# Product Recommender System for Mobility BU, 26-Jul-2019

##### This program using various libraries of Python to gives a recommendation to upsell or cross sell using data of past purchases by customers.
##### For any clarifications contact Surendra Orupalli, surendrao@softcell.com

###### ---------------------------------------------------------------------------------------------------------------------------------------------------------------------

### Collaborative Filtering
Collaborative filtering method was used for creating this recommender system (recsys). Collaborative filtering leverages the power of crowd. The intuition behind collaborative filtering is that if a customer EFG Bank purchased products X and Y, and if another customer LMN Bank purchased product X, there is a fair bit of chance that LMN Bank will buy the product Y as well.

### Similarity Metric
The similarity metric used for collaborative filtering method is correlation coefficient between a product purchased by various customers with all other products purchased by the same set of customers. These correlation coefficients are used to create a correlation matrix. Correlation coefficients and number of customers who have purchased various products have been chosen as the parameters to arrive at a recommended list to products.

### Data Used
The data used here is the all invoice data fromt the time when the Mobility BU started invoicing customer which was from December 2012.


## Fireup Python Libraries

In [22]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns

## Reading the required files

In [23]:
invoice_data = pd.read_excel("/Users/surendraorupalli/Documents/Softcell Apple SBU/Operations/Python Programs/Recommender System/QBR Data_2019-06-30.xlsx")
invoice_data.head(0)

In [24]:
print(invoice_data.head(0))

## Matrix of 'Customer' purchasing various 'Product Code'. Values are sum of Quantity'.

In [25]:
invoice_data_quantity = invoice_data.pivot_table(index="Customer Name", columns="Product Code", values="Quantity")
invoice_data_quantity.head(10)

## Enter a Product Code

In [26]:
prod_code = input("Enter a Product Code: ")

## Drop NaN values

In [27]:
product_code_qty = invoice_data_quantity[prod_code]
product_code_qty_noNA = invoice_data_quantity[prod_code].dropna()
product_code_qty_noNA

## Correlation of Product Code entered with all other Product Codes

In [28]:
products_like_product_code = invoice_data_quantity.corrwith(product_code_qty)

corr_product_code = pd.DataFrame(products_like_product_code, columns=['Correlation'])  
corr_product_code.dropna(inplace=True)  
print(corr_product_code.shape)
corr_product_code.head(10)

In [29]:
corr_product_code_sorted = corr_product_code.sort_values(["Correlation"], ascending=False)
corr_product_code_sorted.head(10)

## Joining correlation with customer counts

In [30]:
customer_count = invoice_data.groupby("Product Code").count()

customer_count_sorted = customer_count.sort_values("Customer Name", ascending=False)
customer_count_sorted.head(10)

In [31]:
corr_product_code_final = pd.merge(corr_product_code_sorted, customer_count_sorted, 
                                   how="left", left_on="Product Code", right_on="Product Code")

corr_product_code_final_001 = corr_product_code_final.sort_values(["Correlation", "Customer Name"], ascending=False)

count_list = corr_product_code_final_001["Customer Name"].tolist()

count_list.sort(reverse=True)
print(count_list)

count_list_topten = count_list[11]
print(count_list_topten)


corr_product_code_final_002 = corr_product_code_final_001[(corr_product_code_final_001["Customer Name"]>count_list_topten) 
                                                            & (corr_product_code_final_001["Correlation"]>0.5)]


corr_product_code_final_002

In [32]:
corr_product_code_final_003 = pd.merge(corr_product_code_final_002, invoice_data, how="left", left_on="Product Code", right_on="Product Code")

## Get Recommended Product Details

In [33]:
# Recommended Products Details
recommended_product_code_list = corr_product_code_final_002.index.tolist()

# Recommended-1 Product Details
recommended_product_code_001 = recommended_product_code_list[0]
recommended_product_name = invoice_data.loc[invoice_data['Product Code'] == recommended_product_code_001, ['Product Name']]
recommended_product_name_001 = recommended_product_name.iloc[0,0]

# Recommended-2 Product Details
recommended_product_code_002 = recommended_product_code_list[1]
recommended_product_name = invoice_data.loc[invoice_data['Product Code'] == recommended_product_code_002, ['Product Name']]
recommended_product_name_002 = recommended_product_name.iloc[0,0]

recommended_data = {'Product Code':[recommended_product_code_001, recommended_product_code_002],
                 'Product Name': [recommended_product_name_001, recommended_product_name_002]}
recommended_table = pd.DataFrame(data=recommended_data)

recommended_table_001 = recommended_table.set_index('Product Code')
recommended_table_001

recommended_table_002 = recommended_table_001.drop(index="MN8X2HN/A")
recommended_table_003 = recommended_table_002.reset_index()

## Get Product Purchased Details

In [34]:
# Purchased Product Details
purchased_product_code = prod_code
purchased = invoice_data.loc[invoice_data['Product Code'] == prod_code, ['Product Name']]
purchased_product_name = purchased.iloc[0,0]

purchased_data = {'Product Code':[purchased_product_code],
                 'Product Name':[purchased_product_name]}
purchased_table = pd.DataFrame(data=purchased_data)

## Product Purchased

In [35]:
# Printing the product product list table without index column
blankIndex=[''] * len(purchased_table)
purchased_table.index=blankIndex
purchased_table

## Products Recommended

In [36]:
# Printing the recommended product list table without index column
blankIndex=[''] * len(recommended_table_003)
recommended_table_003.index=blankIndex
recommended_table_003